# 1. Inicio de Sesión sobre SageMaker

In [6]:
#Utilitario para construir modelos de Machine Learning y Deep Learning
import sagemaker

#Importamos el utilitario para definir el entrenador del algoritmo
from sagemaker.estimator import Estimator

#Librería para manipular los servicios de AWS
import boto3

#Iniciamos sesión en el servicio de SageMaker
sesion = sagemaker.Session()

#Obtenemos la ejecución en donde estamos trabajando
region = sesion.boto_region_name

#Verificamos
print(region)

#Obtenemos el rol de ejecución de SageMaker
#El rol de ejecución permite a SageMaker usar otros servicios de AWS
rol = sagemaker.get_execution_role()

us-east-1


# 2. Lectura de archivos

In [7]:
#Utilitario para leer archivos de datos
from sagemaker.inputs import TrainingInput

#Bucket en donde se encuentran los archivos
#IMPORTANTE: REEMPLAZAR "XXX" POR TUS INICIALES
bucket = "datasetsbdajac"

#Lectura de datos de entrenamiento
dataTrain = TrainingInput(
    f"s3://{bucket}/data/bodyperformance_dataset_validacion_cruzada/vc4/train", #Ruta del archivo
    content_type = "text/csv", #Formato del archivo
    distribution = "FullyReplicated", #El archivo será copiado en todos los servidores
    s3_data_type = "S3Prefix", #Desde donde se lee el archivo (S3)
    input_mode = "File", #Los registros se encuentran dentro de archivos
    record_wrapping = "None" #Envoltorio de optimización
)

#Lectura de datos de validación
dataTest = TrainingInput(
    f"s3://{bucket}/data/bodyperformance_dataset_validacion_cruzada/vc4/test", #Ruta del archivo
    content_type = "text/csv", #Formato del archivo
    distribution = "FullyReplicated", #El archivo será copiado en todos los servidores
    s3_data_type = "S3Prefix", #Desde donde se lee el archivo (S3)
    input_mode = "File", #Los registros se encuentran dentro de archivos
    record_wrapping = "None" #Envoltorio de optimización
)

# 3. Configuración del Algoritmo

In [8]:
#Importamos el utilitario para definir el entrenador del algoritmo
from sagemaker.estimator import Estimator

#Definimos el entrenador del algoritmo
entrenador = Estimator(
    image_uri = sagemaker.image_uris.retrieve("xgboost", region, version = "latest"), #Descargamos la implementación del algoritmo desde la región donde trabajamos
    role = rol, #Rol que ejecuta servicios sobre AWS
    instance_count = 1, #Cantidad de servidores de entrenamiento
    instance_type = "ml.m5.large", #Tipo de servidor de entrenamiento
    predictor_type = "classifier", #Tipo de predicción del algoritmo
    sagemaker_session = sesion, #Sesión de SageMaker
    base_job_name = "entrenamiento-prediccion-categorica", #Nombre del job de entrenamiento
    num_classes = 4 #Número de categorías en el label
)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [9]:
#Configuramos los parametros del algoritmo
entrenador.set_hyperparameters(
    feature_dim = 12, #Cantidad de features
    predictor_type = "classifier", #Indicamos que tipo de predicción es
    normalize_data = "true", #Normalizamos los features
    normalize_label = "false", #NO Normalizamos el label
    num_round = 100, #Número de árboles generados
    max_depth = 7, #Profundidad máxima del arbol
    colsample_bytree = 0.95 #Porcentaje de registros validos para generalizar
)

# 4. Entrenamiento del Modelo

In [10]:
#Entrenamos y validamos el modelo
#MIENTRAS SE ENTRENA EL MODELO: En SageMaker, en la sección "Jobs", en la opción "Training" podemos ver cómo el modelo se entrena
#TIEMPO DE ENTRENAMIENTO: 5 MINUTOS
entrenador.fit({"train": dataTrain, "validation": dataTest})

INFO:sagemaker:Creating training-job with name: entrenamiento-prediccion-categorica-2024-09-30-22-14-41-295


2024-09-30 22:14:43 Starting - Starting the training job...
2024-09-30 22:14:57 Starting - Preparing the instances for training...
2024-09-30 22:15:36 Downloading - Downloading input data......
2024-09-30 22:16:27 Downloading - Downloading the training image...
2024-09-30 22:17:14 Training - Training image download completed. Training in progress.
2024-09-30 22:17:14 Uploading - Uploading generated training modelArguments: train
[2024-09-30:22:17:07:INFO] Running standalone xgboost training.
[2024-09-30:22:17:07:INFO] File size need to be processed in the node: 0.75mb. Available memory size in the node: 240.77mb
[2024-09-30:22:17:07:INFO] Determined delimiter of CSV input is ','
[22:17:07] S3DistributionType set as FullyReplicated
[22:17:07] 10712x12 matrix with 128544 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2024-09-30:22:17:07:INFO] Determined delimiter of CSV input is ','
[22:17:07] S3DistributionType set as FullyReplicated
[22:17:07] 2678x

# 5. ESTADISTICAS DEL MODELO

In [11]:
#Librería para manipular los servicios de AWS
import boto3

#Nos conectamos al servicio de SageMaker
sagemakerCliente = boto3.client("sagemaker")

#Obtenemos el nombre del último entrenamiento realizado
nombreDeEntrenamiento = entrenador.latest_training_job.name

#Verificamos
print(nombreDeEntrenamiento)

#Obtenemos la descripción del entrenamiento
descripcionDeEntrenamiento = sagemakerCliente.describe_training_job(TrainingJobName = nombreDeEntrenamiento)

#Vemos toda la descripción de entrenamiento
descripcionDeEntrenamiento

#Dentro de la sub-variable "FinalMetricDataList" tenemos las métricas del modelo
#El error cuadrático medio se encuentra en "validation:mse"
#El r2 (error "porcentual") se encuentra en "validation:r2"
#IMPORTANTE: En SageMaker, en la sección "Jobs", en la opción "Training" podemos ver las estadísticas del modelo
descripcionDeEntrenamiento["FinalMetricDataList"]

entrenamiento-prediccion-categorica-2024-09-30-22-14-41-295


[{'MetricName': 'train:rmse',
  'Value': 0.24172300100326538,
  'Timestamp': datetime.datetime(2024, 9, 30, 22, 17, 9, tzinfo=tzlocal())},
 {'MetricName': 'validation:rmse',
  'Value': 0.5998409986495972,
  'Timestamp': datetime.datetime(2024, 9, 30, 22, 17, 9, tzinfo=tzlocal())}]